In [11]:
import pandas as pd
import numpy as np 
import os
import plotly.express as px
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from matplotlib import pyplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
#from sklearn.metrics import mean_absolute_percentage_errors
import random

# For investigating timeseries data
from sklearn import preprocessing
from sklearn.model_selection import ParameterGrid
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose

# For modeling
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from prophet import Prophet

### Reading Data

In [12]:
# Reading Data
base_path =  os.getcwd()
file_name = 'Traffic_Data.xlsx'
total_path = base_path + '//Data//' 
df = pd.read_excel(total_path + file_name, sheet_name='Sheet1')
df.head(10)


,State,Region,STATIONS,CMILES,PMILES,Month,Month_2,Year,Date
0,Connecticut,Northeast,14,2546,2432,November,11,2023,2023-11-01
1,Maine,Northeast,130,1177,1148,November,11,2023,2023-11-01
2,Massachusetts,Northeast,227,5148,5013,November,11,2023,2023-11-01
3,New Hampshire,Northeast,150,1062,1034,November,11,2023,2023-11-01
4,New Jersey,Northeast,73,6569,6339,November,11,2023,2023-11-01
5,New York,Northeast,110,9144,8825,November,11,2023,2023-11-01
6,Pennsylvania,Northeast,57,8610,8408,November,11,2023,2023-11-01
7,Rhode Island,Northeast,26,661,659,November,11,2023,2023-11-01
8,Vermont,Northeast,35,543,531,November,11,2023,2023-11-01
9,Delaware,South Atlantic,0,923,900,November,11,2023,2023-11-01


### Filtering for States

In [13]:
states_of_interest = ['Oregon', 'Washington', 'California']

# Filtering for states of interest
df = df[df['State'].isin(states_of_interest)]
df = df.sort_values(by = ['Date']).reset_index()
df.head(10)

for val in states_of_interest:
    # Replacing outlier January 2023 value
    mean_val = df[(df['Month'] == 'January') &
              (df['Year'] != 2023) &
              (df['State'] == val)]['CMILES'].mean()
    df['CMILES'] = np.where((df['Month'] == 'January') & (df['Year'] == 2023) & (df['State'] == val), mean_val, df['CMILES'])

### Plotting Data

In [14]:
fig = px.scatter(df.reindex(), x="Date", y="CMILES", color = 'State',
                  trendline="lowess",trendline_options=dict(frac=0.1),
                  title = 'Miles Driven by Time - Oregon')
fig.show()

## Fitting Prophet Model

Now we'll fit prophet models.

In [ ]:
def evaluate_prophet(df, states_of_interest, window = 4):

    '''
    Trains prophet model using input dataframe for each of the specified factor levels
    and forecasts out a number of timesteps defined by the window input.
    '''
    
    models = {}
    for val in states_of_interest:

        # Generating series for 1 factor level
        series_of_interst = df[df['State'] == val].rename(columns= {'Date': 'ds', 'CMILES': 'y'})[['ds', 'y']]

        # Fitting model for series
        my_model = Prophet()
        my_model.fit(series_of_interst)

        # Generating future dataset and makign predictions
        future_dates = my_model.make_future_dataframe(periods=window, freq='month')
        forecast = my_model.predict(future_dates)

        # Plotting results
        my_model.plot(forecast, uncertainty=True)

        # saving model
        models['val'] = my_model
    return models